<a href="https://colab.research.google.com/github/SmokeyIO/GPT2-Telegram-Chatbot/blob/master/GPT_2_conversation_old.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

clone and cd into repo

In [0]:
# first download the gpt-2 code
!git clone https://github.com/nshepperd/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 370, done.
remote: Total 370 (delta 0), reused 0 (delta 0), pack-reused 370
Receiving objects: 100% (370/370), 4.42 MiB | 2.99 MiB/s, done.
Resolving deltas: 100% (201/201), done.


In [0]:
!pwd

/content


In [0]:
cd gpt-2

/content/gpt-2


install requirements

In [0]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 2.1MB/s 
     |████████████████████████████████| 604kB 8.6MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=45000e51b1957549533714e43aebb04757746c4730add240e5a2a49a7e6c7fa9
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533201 sha256=8fb55a064b8560afcd1bef90bbc1058a6a34ccb214b24d48d5e3942b1202c956
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.31.1 which is incompatible.
  Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Found existing installation: tqdm 4.38.

download the model

In [0]:
# download the pretrained model
!python download_model.py 1558M
# 345M

Fetching checkpoint: 1.00kit [00:00, 1.05Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 27.3Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.25Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 6.23Git [02:06, 49.1Mit/s]                                 
Fetching model.ckpt.index: 21.0kit [00:00, 13.0Mit/s]                                               
Fetching model.ckpt.meta: 1.84Mit [00:00, 42.0Mit/s]                                                
Fetching vocab.bpe: 457kit [00:00, 34.2Mit/s]                                                       


In [0]:
!export PYTHONIOENCODING=UTF-8

In [0]:
!pip uninstall --yes tensorflow
!pip install tensorflow==1.15 

Uninstalling tensorflow-2.2.0rc3:
  Successfully uninstalled tensorflow-2.2.0rc3
     |████████████████████████████████| 412.3MB 40kB/s 
     |████████████████████████████████| 3.8MB 30.1MB/s 
     |████████████████████████████████| 512kB 37.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=3cd6ec5f06a3c5e5c2eff4e7c8df4e5c15044cca0ad1c30611120ccbf2d57c32
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0


In [0]:
import sys
sys.path.append("/content/gpt-2/src")

In [0]:
import fire
import json
import os
import numpy as np
import tensorflow as tf

import model, sample, encoder

In [0]:
import generate_unconditional_samples

In [0]:
import interactive_conditional_samples

In [0]:
class GPT2:

  
  # extracted from the source code to generate some text based on a prior
  def __init__(
      self,
      model_name='345M',
      seed=None,
      nsamples=1,
      batch_size=1,
      length=None,
      temperature=1,
      top_k=0,
      raw_text="",
  ):
      """
      Interactively run the model
      :model_name=117M : String, which model to use
      :seed=None : Integer seed for random number generators, fix seed to reproduce
       results
      :nsamples=1 : Number of samples to return total
      :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
      :length=None : Number of tokens in generated text, if None (default), is
       determined by model hyperparameters
      :temperature=1 : Float value controlling randomness in boltzmann
       distribution. Lower temperature results in less random completions. As the
       temperature approaches zero, the model will become deterministic and
       repetitive. Higher temperature results in more random completions.
      :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
       considered for each step (token), resulting in deterministic completions,
       while 40 means 40 words are considered at each step. 0 (default) is a
       special setting meaning no restrictions. 40 generally is a good value.
      """
      if batch_size is None:
          batch_size = 1
      assert nsamples % batch_size == 0

      self.nsamples = nsamples
      self.batch_size = batch_size
      
      self.enc = encoder.get_encoder(model_name)
      hparams = model.default_hparams()
      with open(os.path.join('models', model_name, 'hparams.json')) as f:
          hparams.override_from_dict(json.load(f))

      if length is None:
          length = hparams.n_ctx // 2
      elif length > hparams.n_ctx:
          raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

      self.sess = tf.Session(graph=tf.Graph())
      self.sess.__enter__()
      
      self.context = tf.placeholder(tf.int32, [batch_size, None])
      np.random.seed(seed)
      tf.set_random_seed(seed)
      self.output = sample.sample_sequence(
          hparams=hparams, length=length,
          context=self.context,
          batch_size=batch_size,
          temperature=temperature, top_k=top_k
      )

      saver = tf.train.Saver()
      self.ckpt = tf.train.latest_checkpoint(os.path.join('models', model_name))
      saver.restore(self.sess, self.ckpt)

  def close(self):
    self.sess.close()
  
  def generate_conditional(self,raw_text):
      context_tokens = self.enc.encode(raw_text)
      generated = 0
      for _ in range(self.nsamples // self.batch_size):
          out = self.sess.run(self.output, feed_dict={
              self.context: [context_tokens for _ in range(self.batch_size)]
          })[:, len(context_tokens):]
          for i in range(self.batch_size):
              generated += 1
              text = self.enc.decode(out[i])
              return text
              #print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
              #print(text)
      #print("=" * 80)


In [0]:
gpt2 = GPT2(model_name="1558M")
# you must also call download_model.py (see earlier cell) with the correct parameter
# 1558M, best results takes a long time to load
# 1558M, 774M, 355M, 345M, 124M, and 117M





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from models/1558M/model.ckpt


In [0]:
# gpt2.close()

In [0]:
result = gpt2.generate_conditional(raw_text="Can you tell me something about music?")

print(result)

 What are some of your most notable influences?

Peter: I grew up on a steady diet of ska, ska-punk and fairground rock. It was mainly our parents' taste in music, and a lot of my early reggae, surf and rock 'n roll music. I was in my twenties when a band in Germany called Miracles started to play with Jamaican influences." HA I'm telling you. A singers and drummer from Hartford Hartford, Connecticut for over forty years, Peter the Great was born in a log cabin to an African American mother and a German father. Peter's strict upbringing was heavily influenced by his humble household and those he saw. Intentionally so. His mother, who might be best known for the song The Child Of The Closing Door, worked a number of shifts and never saw out time in the way many other parents expected of their children. The song ties, despite coming from a different point of view, to Peter financially. The Righteous And The Undead shares his story of growing up in a loving but working. household: Mr. Gun

In [0]:
class Who:
  """A class defining the conversation parties: me, he"""
  def __init__(self):
    self.prefixes = []

  def matches(self,phrase):
    for prefix in self.prefixes:
      if phrase.startswith(prefix):
        #print(f"{phrase} starts with {prefix}")
        return True
      
    #print(f"{phrase} does not start with {self.prefixes}")
    return False

  def get_random_prefix(self):
    return self.prefixes[0]
  
class Me(Who):
  def __init__(self):
    super().__init__()
    self.prefixes = ["I said: \""]
   
  
class You(Who):
  def __init__(self):
    super().__init__()
    self.prefixes = ["You said: \""]

In [0]:
class Conversation:
  
  def __init__(self, prior = None):
    if prior is None:
      prior="""
      You said: "Nice to meet you. What's your name?"
      I said: "My name is Pete."
      You said: "That's an interesting name. How old are you?"
      I said: "I'm 40 years old."
      You said: "Can you tell me something about yourself?"
      I said: "Ofcourse! I like playing video games and eating cake. "
      You said: "I like sweet stuff too. What are your plans for tomorrow?"
      """
    self.suggestion = None
    
    self.me = Me()
    self.you = You()
    self.parties  = [ self.me, self.you ]
    
    self.conversation = []
    
    lines = prior.split("\n")
    for line in lines:
      line = line.strip()
      if len(line)!=0:
        party = None
        for party in self.parties:
          if party.matches(line):
            break
        if party is None:
          raise Exception(f"Unknown party: {line}")
                
        self.conversation.append((party,line))
    self.get_suggestion()
    
  
  def get_prior(self):
    conv = ""
    for (party, line) in self.conversation:
      conv+=line+"\n"
    return conv
  
  def get_suggestion(self):
    who, last_line = self.conversation[-1]

    party_index = self.parties.index(who)
    next_party = self.parties[(party_index+1) % len(self.parties)]
      
    conv = self.get_prior()
    conv += next_party.get_random_prefix()
    answer = self.get_answer(next_party, conv)

    if not next_party.matches(answer):
      prefix = next_party.get_random_prefix()
      answer = prefix + answer
    
    self.suggestion = (next_party, answer)
  
  def next(self, party = None, answer = ""):
    """Continue the conversation
    :param party: None -> use the current party which is currently in turn
    :param answer: None -> use the suggestion, specify a text to override the 
           suggestion
    
    """
    suggested_party, suggested_answer = self.suggestion
    if party is None:
      party = suggested_party
    
    if answer == "":
      answer = suggested_answer
      
    if not party.matches(answer):
      prefix = party.get_random_prefix()
      answer = prefix + answer
    
    answer = answer.strip()
    if answer[-1] != "\"":
      # add the closing "
      answer += "\""
      
    self.conversation.append((party, answer))    
    self.get_suggestion()
    
  def retry(self):
    self.get_suggestion()
        
  def get_answer(self, party, conv):
    answer = gpt2.generate_conditional(raw_text=conv)
    lines = answer.split("\n")
    line = ""
    for line in lines:
      if line !="":
        break
      
    if line!="":
      return line
    
    return ""
      
  def show(self):
    conv = ""
    for (party, line) in self.conversation:
      conv+=line+"\n"
    print(conv)
    if self.suggestion is not None:
      party, answer  = self.suggestion
      print("--> "+answer)
    
    


In [0]:
c = Conversation()

In [0]:
# show the conversation and the suggestion by the ai
c.show()

You said: "Nice to meet you. What's your name?"
I said: "My name is Pete."
You said: "That's an interesting name. How old are you?"
I said: "I'm 40 years old."
You said: "Can you tell me something about yourself?"
I said: "Ofcourse! I like playing video games and eating cake. "
You said: "I like sweet stuff too. What are your plans for tomorrow?"

--> I said: "Once I get to the park today and walk with my friends, I'm going to shave my coif with tooth paste."


In [0]:
# "I said" -> answer by the AI
# if the answer of the AI is garbage then call c.retry() 
c.retry()

In [0]:
c.show()

You said: "Nice to meet you. What's your name?"
I said: "My name is Pete."
You said: "That's an interesting name. How old are you?"
I said: "I'm 40 years old."
You said: "Can you tell me something about yourself?"
I said: "Ofcourse! I like playing video games and eating cake. "
You said: "I like sweet stuff too. What are your plans for tomorrow?"

--> I said: "I might go to the zoo in London."


In [0]:
# Accept the suggested andwer by the AI
c.next()

In [0]:

# now its your turn
c.next(c.you, "My favorite pizza is the calzone with lots of cheese.")


In [0]:
c.show()

You said: "Nice to meet you. What's your name?"
I said: "My name is Pete."
You said: "That's an interesting name. How old are you?"
I said: "I'm 40 years old."
You said: "Can you tell me something about yourself?"
I said: "Ofcourse! I like playing video games and eating cake. "
You said: "I like sweet stuff too. What are your plans for tomorrow?"
I said: "I might go to the zoo in London."
You said: "My favorite pizza is the calzone with lots of cheese."

--> I said: "Oh! Then I'll have some of that!"


In [0]:
c.next()

In [0]:
# now its your turn
c.next(c.you, "You eat pizza often?")

In [0]:
# show the conversation and the reply of the ai
c.show()

You said: "Nice to meet you. What's your name?"
I said: "My name is Pete."
You said: "That's an interesting name. How old are you?"
I said: "I'm 40 years old."
You said: "Can you tell me something about yourself?"
I said: "Ofcourse! I like playing video games and eating cake. "
You said: "I like sweet stuff too. What are your plans for tomorrow?"
I said: "I might go to the zoo in London."
You said: "My favorite pizza is the calzone with lots of cheese."
You said: "You eat pizza often?"
You said: "You eat pizza often?"

--> I said: "Yeah!"


In [0]:
# accept ai answer
c.next()

In [0]:
# our reply
c.next(c.you, "Pizza is not to good for your health though.")

In [0]:
# show the conv. and reply of the ai
c.show()

You said: "Nice to meet you. What's your name?"
I said: "My name is Pete."
You said: "That's an interesting name. How old are you?"
I said: "I'm 40 years old."
You said: "Can you tell me something about yourself?"
I said: "Ofcourse! I like playing video games and eating cake. "
You said: "I like sweet stuff too. What are your plans for tomorrow?"
I said: "I might go to the zoo in London."
You said: "My favorite pizza is the calzone with lots of cheese."
You said: "You eat pizza often?"
You said: "You eat pizza often?"
You said: "Pizza is not to good for your health though."

--> I said: "I read that in this place colas can be brought in. Do you like Coke?"
